In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [2]:
import torch
import pytorch_lightning as pl
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pytorch_lightning.loggers import TensorBoardLogger
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
from systems.Fields.scalar_field import Scalar_Field
from systems.Fields.phi4 import Phi4
from Data import MY_Dataset
from flows.NormalizingFlow import NormalizingFlow    
from lattice import Lattice

def set_random_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)

set_random_seed(42)


latt = Lattice(np.array([8,8]),np.array([16,4]))




In [4]:
normal_dist = torch.distributions.Normal(loc = torch.zeros(latt.total_nodes), scale = torch.ones(latt.total_nodes))
DS=MY_Dataset(normal_dist,epoch_size=2**16)
train_loader = DataLoader(DS, batch_size=2**12, shuffle=True)

In [5]:
class Pipeline(pl.LightningModule):
    def __init__(
        self,
        model,
        latent,
        criterion,
        optimizer_class=torch.optim.Adam,
        optimizer_kwargs={"lr": 0.001,"weight_decay": 0}
    ) -> None:
        super().__init__()
        self.model = model
        self.loss = criterion
        self.latent = latent
        self.optimizer_class = optimizer_class
        self.optimizer_kwargs = optimizer_kwargs

    """
    def configure_optimizers(self):
        optimizer = self.optimizer_class(
            self.model.parameters(), **self.optimizer_kwargs
        )
        return optimizer
    """
    
    def configure_optimizers(self):
        optimizer = self.optimizer_class(self.model.parameters(), **self.optimizer_kwargs)
        
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer=optimizer,
            min_lr=1e-5,
            factor=0.95,
            mode="min",
            verbose=True,
        )

        lr_scheduler = {
            "scheduler": scheduler,
            "interval": "epoch",
            "monitor": "train_loss",
        }
        return [optimizer], [lr_scheduler]

    def training_step(self, batch, batch_idx):
        z = batch
        latent_log_prob = torch.sum(self.latent.log_prob(z),(1,2))
        
        x, log_abs_det = self.model.g(z)
        loss = self.loss(x,log_abs_det)
        ess = self.loss.ESS(latent_log_prob,log_abs_det)

        
        sch = self.lr_schedulers()
        sch.step(loss)
        self.log('train_loss', loss,prog_bar=True)
        self.log('ess',ess)
        self.log('mean_x',torch.mean(x))
        self.log('lr',sch.get_last_lr()[0])
        return loss


    def on_train_epoch_end(self):
        #print("---------------------------end epoch---------------------------------")
        pass
    
    def on_validation_end(self) -> None:
        if not self.automatic_optimization:
            # Save a checkpoint of the model
            ckpt_path = os.path.join(self.trainer.log_dir, 'checkpoints', 'ckpt.pt')
            self.trainer.save_checkpoint(ckpt_path, weights_only=True)
        return super().on_validation_end()
    
    

In [6]:
set_random_seed(42)
rg = NormalizingFlow.config_and_init(n_flows = 4,num_hidden = 6,hidden_dim = 2 * latt.total_nodes,lattice=latt,ort=True)
print(len(rg.flows))
rg.save("./weights/first.pth") 

4


In [7]:
rg = NormalizingFlow.load_model("./weights/first.pth")

In [8]:
z = next(iter(train_loader))
#print(z.shape)
x = rg.g(z)

In [17]:
def get_pair_split_masks_field(lattice,dir):
    bigmask = list(range(lattice.total_nodes))
    mask1 = list(filter(lambda x: x% (4 * lattice.n_nodes[dir] ** dir) < 2 * lattice.n_nodes[dir] ** dir,bigmask))       
    mask2 = list(filter(lambda x: x%4>=2,bigmask))
    return [mask1, mask2]

In [18]:
print(latt.n_nodes[0])
mask=get_pair_split_masks_field(latt,1)
print(mask)

8
[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47], [2, 3, 6, 7, 10, 11, 14, 15, 18, 19, 22, 23, 26, 27, 30, 31, 34, 35, 38, 39, 42, 43, 46, 47, 50, 51, 54, 55, 58, 59, 62, 63]]


In [ ]:
set_random_seed(42)
phi4.set_J_local(1,16)
phi4.mass2 = 1
for i in range(len(beta_arr)):
    
    phi4.beta = beta_arr[i]
    pipeline = Pipeline(model = rg, 
                  latent = normal_dist ,
                  criterion = phi4.get_KL(), 
                  optimizer_class=torch.optim.Adam,
                  optimizer_kwargs={"lr": 0.001,"weight_decay":0.0})

    trainer = pl.Trainer(
        max_epochs = 15,
        logger = TensorBoardLogger(save_dir=f"./logs/field_phi4_beta"),
        num_sanity_val_steps = 0,
        log_every_n_steps = 1,
        enable_checkpointing = False,
        accumulate_grad_batches = 1)

    trainer.fit(model=pipeline, train_dataloaders=train_loader)
    rg.save("./weights/field/phi4_beta_jloc/phi4_beta_jloc"+str(beta_arr[i])+".pth")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
C:\Users\SEVA1\anaconda3\Lib\site-packages\torch\optim\lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "

  | Name  | Type      | Params | Mode 
--------------------------------------------
0 | model | RGflows   | 3.0 M  | train
1 | loss  | KL_with_S | 0      | train
--------------------------------------------
3.0 M     Trainable params
0         Non-trainable params
3.0 M     Total params
11.993    Total estimated model params size (MB)
C:\Users\SEVA1\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve 

Training: |                                                                                      | 0/? [00:00<…

C:\Users\SEVA1\anaconda3\Lib\site-packages\pytorch_lightning\trainer\call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type      | Params | Mode 
--------------------------------------------
0 | model | RGflows   | 3.0 M  | train
1 | loss  | KL_with_S | 0      | train
--------------------------------------------
3.0 M     Trainable params
0         Non-trainable params
3.0 M     Total params
11.993    Total estimated model params size (MB)


Training: |                                                                                      | 0/? [00:00<…

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type      | Params | Mode 
--------------------------------------------
0 | model | RGflows   | 3.0 M  | train
1 | loss  | KL_with_S | 0      | train
--------------------------------------------
3.0 M     Trainable params
0         Non-trainable params
3.0 M     Total params
11.993    Total estimated model params size (MB)


Training: |                                                                                      | 0/? [00:00<…

In [48]:
set_random_seed(42)
scalar.set_J_local(4,16)
phi4.set_J_local(4,16)
pipeline=Pipeline(model=rg, 
                  latent = normal_dist ,
                  criterion = phi4.get_KL(), 
                  optimizer_class=torch.optim.Adam,
                  optimizer_kwargs={"lr": 0.001,"weight_decay":0.0})

trainer = pl.Trainer(
    max_epochs = 2000,
    logger = TensorBoardLogger(save_dir=f"./logs/field"),
    num_sanity_val_steps = 0,
    log_every_n_steps = 1,
    enable_checkpointing = False,
    accumulate_grad_batches = 1
)

trainer.fit(model=pipeline, train_dataloaders=train_loader)
rg.save("./weights/field/phi4_beta_jloc/phi4_beta_jloc16.pth")

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type      | Params | Mode 
--------------------------------------------
0 | model | RGflows   | 3.0 M  | train
1 | loss  | KL_with_S | 0      | train
--------------------------------------------
3.0 M     Trainable params
0         Non-trainable params
3.0 M     Total params
11.993    Total estimated model params size (MB)


Training: |                                                                                      | 0/? [00:00<…

In [4]:
import torch
from transforms import t_get_O
mat = t_get_O(8)
mat_t = torch.t(mat)
ons = torch.einsum("sj,js->s",mat,mat_t) 
print(ons)

tensor([1., 1., 1., 1., 1., 1., 1., 1.])
